# Episodes of CartPole implemented in MAPDL   

In [1]:
!pip install ./pyansys_rl ./pyansys_gym -q --user --no-warn-script-location

ERROR: Could not find a version that satisfies the requirement tensorflow==1.13.1 (from pyansys-rl==0.0.1) (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.8.0rc0, 2.8.0rc1)
ERROR: No matching distribution found for tensorflow==1.13.1 (from pyansys-rl==0.0.1)


In [2]:
import itertools
import os

import gym
import numpy as np

import pyansys_cartpole

np.set_printoptions(precision=4, suppress=True)

ModuleNotFoundError: No module named 'pyansys_cartpole'

<h2>Background:  Markov Decision Process</h2>
<img src="media/MDP_board.jpg" alt="Drawing" style="width: 400px;"/>

In a Markov Decision Process we have an agent immersed in an environment.  At any given time, the agent finds itself in a state and it must select one of the available actions.  Upon taking an action, the environment reponds by assigning a reward and transitioning the agent to a successor state.  This loop continues until a terminal state is reached.  It is interesting to ask: could we learn to act optimally in such a setup? could we learn to select sequences of actions that maximize long term cumulative rewards? 

<img src="media/MDP_loop.jpg" alt="Drawing" style="width: 700px;"/>

<h2>CartPole</h2>
<img src="media/cartpole_description.jpg" alt="Drawing" style="width: 600px;"/>

The CartPole is a classic control problem.  It is a balancing task: push the cart such that the pinned pole remains upright. In other words, the pole behaves as a solid inverted pendulum and is unstable about the desired configuration.  A simple implementation could use a revolute/hinge joint between the cart and the pole, and a translational joint between the cart and the ground. 

<h3>MAPDL in the loop</h3>
<img src="media/ANSYS_loop.jpg" alt="Drawing" style="width: 600px;"/>
<center>Fig: A single iteration of the CartPole as a Markov Decision Process using MAPDL</center>

In this implementation of the CartPole as a Markov Decision Process, we highlight the following components:

* Actions: push either left (0) or right (1) 
* State: $x_{\text{cart}}, v_{\text{cart}}, \theta_{\text{pole}}, v_{\text{pole}}$
* Reward: +1 for every timestep still in equilibrium
* Transition Model: courtesy of an MAPDL structural transient analysis 

At each episode, the system starts in a randomly seeded state, with positions, velocities and angles picked from a uniform distribution about the vertical/resting position, thus it is unlikely to ever be at equilibrium.  Even if it were, the equilibrium would be unstable. 

## Instance creation: MAPDL in the loop
Create an instance of an MAPDL environment that is specially wrapped for use in [OpenAI Gym](https://gym.openai.com/) thanks to the newly developed python gRPC bindings ([pyansys](https://pypi.org/project/pyansys/)).  The wrapper sets up the CartPole physics, accepts the available actions (i.e. forces), and calculates the state transitions (kinematic response) after every time step (an MAPDL load step).  For reference, OpenAI Gym provides its own ad hoc [environment](https://gym.openai.com/envs/CartPole-v1/) for solving the system's [kinematic equations](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)

In [ ]:
env_name = 'pyansys-CartPole-v0'
env = gym.make(env_name)

run several episodes (e.g., 3) of the CartPole using a random action, i.e., sometimes 0 (push left), sometimes 1 (push right)

In [ ]:
n_episodes = 3
for i in range(n_episodes):
    print('*' * 30, f'Episode: {i+1}', '*' * 30)
    cur_state = env.reset()
    done, r_tot = False, 0
    while not done:
        action = np.random.choice([0, 1])
        next_state, reward, done, info = env.step(action)
        print('State:', cur_state, '\tAction:', '--->' if action else '<---', '\tReward: ', reward)
        cur_state, r_tot = next_state, r_tot + reward
    print('Episode Reward:', r_tot)
    print('')